In [1]:
#Import all the important files 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# creating a data frame from CSV
customer = pd.read_csv("Customer.csv")
prod_cat_info = pd.read_csv("prod_cat_info.csv")
transactions=pd.read_csv("Transactions.csv")


In [3]:
# changing column label to similar in all tables
customer.rename(columns = {'customer_Id':'cust_id'}, inplace = True)
# removing row if customer_id/prod_sub_cat_code is null from customer and prod_cat_info dataframe
customer.dropna(subset=['cust_id'],inplace=True)
prod_cat_info.dropna(subset=['prod_sub_cat_code'],inplace=True)

In [4]:
#display top 5 rows of customer data
customer.head(5)

,cust_id,DOB,Gender,city_code
0,268408,02-01-1970,M,4.0
1,269696,07-01-1970,F,8.0
2,268159,08-01-1970,F,8.0
3,270181,10-01-1970,F,2.0
4,268073,11-01-1970,M,1.0


In [5]:
#display top 5 rows of prod_cat_info data
prod_cat_info.head(5)

,prod_cat_code,prod_cat,prod_sub_cat_code,prod_subcat
0,1,Clothing,4,Mens
1,1,Clothing,1,Women
2,1,Clothing,3,Kids
3,2,Footwear,1,Mens
4,2,Footwear,3,Women


In [6]:
#display top 5 rows of transactions data
transactions.head(5)

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,80712190438,270351,28/02/14,1,1,-5,-772,405.300,-4265.300,e-Shop
1,29258453508,270384,27/02/14,5,3,-5,-1497,785.925,-8270.925,e-Shop
2,51750724947,273420,24/02/14,6,5,-2,-791,166.110,-1748.110,TeleShop
3,93274880719,271509,24/02/14,11,6,-3,-1363,429.345,-4518.345,e-Shop
4,51750724947,273420,23/02/14,6,5,-2,-791,166.110,-1748.110,TeleShop


In [7]:
#display rows and columns of customer data
customer.shape

(5647, 4)

In [8]:
#display rows and columns of prod_cat_info data
prod_cat_info.shape

(23, 4)

In [9]:
#display rows and columns of transactions data
transactions.shape

(23053, 10)

In [10]:
# creating super table by joining customer and prod_cat_info to transactions
df1=transactions.merge(customer,how='left',on='cust_id')
transaction_master_bi=pd.merge(left=df1, right=prod_cat_info,how='left',left_on=['prod_cat_code','prod_subcat_code'],right_on=['prod_cat_code','prod_sub_cat_code'])
transaction_master_bi.drop(columns='prod_sub_cat_code',axis=1,inplace=True)


In [11]:
transaction_master_bi.head(10)

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,DOB,Gender,city_code,prod_cat,prod_subcat
0,80712190438,270351,28/02/14,1,1,-5,-772,405.300,-4265.300,e-Shop,26-09-1981,M,5.0,Clothing,Women
1,29258453508,270384,27/02/14,5,3,-5,-1497,785.925,-8270.925,e-Shop,11-05-1973,F,8.0,Electronics,Computers
2,51750724947,273420,24/02/14,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0,Books,DIY
3,93274880719,271509,24/02/14,11,6,-3,-1363,429.345,-4518.345,e-Shop,08-06-1981,M,3.0,Home and kitchen,Bath
4,51750724947,273420,23/02/14,6,5,-2,-791,166.110,-1748.110,TeleShop,27-07-1992,M,8.0,Books,DIY
5,97439039119,272357,23/02/14,8,3,-2,-824,173.040,-1821.040,TeleShop,09-10-1982,F,6.0,Electronics,Personal Appliances
6,45649838090,273667,22/02/14,11,6,-1,-1450,152.250,-1602.250,e-Shop,29-05-1981,M,9.0,Home and kitchen,Bath
7,22643667930,271489,22/02/14,12,6,-1,-1225,128.625,-1353.625,TeleShop,21-04-1971,M,9.0,Home and kitchen,Tools
8,79792372943,275108,22/02/14,3,1,-3,-908,286.020,-3010.020,MBR,04-11-1971,F,8.0,Clothing,Kids
9,50076728598,269014,21/02/14,8,3,-4,-581,244.020,-2568.020,e-Shop,27-11-1979,F,3.0,Electronics,Personal Appliances


In [12]:
transaction_master_bi.shape

(23053, 15)

In [13]:
transaction_master_bi.columns

Index(['transaction_id', 'cust_id', 'tran_date', 'prod_subcat_code',
       'prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt', 'Store_type', 'DOB',
       'Gender', 'city_code', 'prod_cat', 'prod_subcat'],
      dtype='object')

In [14]:
transaction_master_bi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23053 entries, 0 to 23052
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    23053 non-null  int64  
 1   cust_id           23053 non-null  int64  
 2   tran_date         23053 non-null  object 
 3   prod_subcat_code  23053 non-null  int64  
 4   prod_cat_code     23053 non-null  int64  
 5   Qty               23053 non-null  int64  
 6   Rate              23053 non-null  int64  
 7   Tax               23053 non-null  float64
 8   total_amt         23053 non-null  float64
 9   Store_type        23053 non-null  object 
 10  DOB               23053 non-null  object 
 11  Gender            23044 non-null  object 
 12  city_code         23045 non-null  float64
 13  prod_cat          23053 non-null  object 
 14  prod_subcat       23053 non-null  object 
dtypes: float64(3), int64(6), object(6)
memory usage: 2.6+ MB


In [15]:
transaction_master_bi.describe()

,transaction_id,cust_id,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,city_code
count,2.305300e+04,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000,23053.000000,23045.000000
mean,5.007348e+10,271021.746497,6.149091,3.763632,2.432395,636.369713,248.667192,2107.308002,5.482534
std,2.898194e+10,2431.692059,3.726372,1.677016,2.268406,622.363498,187.177773,2507.561264,2.863499
min,3.268991e+06,266783.000000,1.000000,1.000000,-5.000000,-1499.000000,7.350000,-8270.925000,1.000000
25%,2.493864e+10,268935.000000,3.000000,2.000000,1.000000,312.000000,98.280000,762.450000,3.000000
50%,5.009313e+10,270980.000000,5.000000,4.000000,3.000000,710.000000,199.080000,1754.740000,5.000000
75%,7.533000e+10,273114.000000,10.000000,5.000000,4.000000,1109.000000,365.715000,3569.150000,8.000000
max,9.998755e+10,275265.000000,12.000000,6.000000,5.000000,1500.000000,787.500000,8287.500000,10.000000


In [16]:
transaction_master_bi.isnull().sum()

transaction_id      0
cust_id             0
tran_date           0
prod_subcat_code    0
prod_cat_code       0
Qty                 0
Rate                0
Tax                 0
total_amt           0
Store_type          0
DOB                 0
Gender              9
city_code           8
prod_cat            0
prod_subcat         0
dtype: int64

In [17]:
# correceting data type of numeric and date columns
transaction_master_bi['city_code']=abs(pd.to_numeric(transaction_master_bi['city_code'],downcast='integer').fillna(1).astype(int))
transaction_master_bi['tran_date'] = pd.to_datetime(transaction_master_bi['tran_date'])
transaction_master_bi['DOB'] = pd.to_datetime(transaction_master_bi['DOB'])
transaction_master_bi['Qty']=abs(transaction_master_bi['Qty'])
transaction_master_bi['Rate']=abs(transaction_master_bi['Rate'])
transaction_master_bi['total_amt']=abs(transaction_master_bi['total_amt'])

/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90613/3146049722.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transaction_master_bi['tran_date'] = pd.to_datetime(transaction_master_bi['tran_date'])
/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90613/3146049722.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  transaction_master_bi['DOB'] = pd.to_datetime(transaction_master_bi['DOB'])


In [18]:
# handling null values
transaction_master_bi.dropna(subset=['transaction_id','cust_id','prod_subcat_code','prod_cat_code','tran_date'],inplace=True)

In [19]:
transaction_master_bi['Gender'].fillna('F',inplace=True)
transaction_master_bi['Store_type'].fillna('NA',inplace=True)
transaction_master_bi['city_code'].fillna(-1,inplace=True)
transaction_master_bi['DOB'].fillna(pd.to_datetime('1900-01-01'),inplace=True)


In [20]:
transaction_master_bi['Qty'].fillna(1,inplace=True)
transaction_master_bi['Qty'] = transaction_master_bi['Qty'].apply(lambda x: 1 if x == 0 else x)
transaction_master_bi['Rate'].fillna(transaction_master_bi['Rate'].mean(),inplace=True)
avg_tax=transaction_master_bi['Tax'].mean()
amount_before_tax=transaction_master_bi['Rate']*transaction_master_bi['Qty']
avg_amount=amount_before_tax.mean()
avg_tax_rate=avg_tax/avg_amount
avg_tax=avg_tax_rate*avg_amount
transaction_master_bi['Tax'].fillna(avg_tax,inplace=True)
total_amount=transaction_master_bi['Rate']*transaction_master_bi['Qty']+transaction_master_bi['Tax']
avg_total_amount=total_amount.mean()
transaction_master_bi['total_amt'].fillna(avg_total_amount,inplace=True)



In [21]:
transaction_master_bi

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,DOB,Gender,city_code,prod_cat,prod_subcat
0,80712190438,270351,2014-02-28,1,1,5,772,405.300,4265.300,e-Shop,1981-09-26,M,5,Clothing,Women
1,29258453508,270384,2014-02-27,5,3,5,1497,785.925,8270.925,e-Shop,1973-05-11,F,8,Electronics,Computers
2,51750724947,273420,2014-02-24,6,5,2,791,166.110,1748.110,TeleShop,1992-07-27,M,8,Books,DIY
3,93274880719,271509,2014-02-24,11,6,3,1363,429.345,4518.345,e-Shop,1981-06-08,M,3,Home and kitchen,Bath
4,51750724947,273420,2014-02-23,6,5,2,791,166.110,1748.110,TeleShop,1992-07-27,M,8,Books,DIY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23048,94340757522,274550,2011-01-25,12,5,1,1264,132.720,1396.720,e-Shop,1972-02-21,M,7,Books,Academic
23049,89780862956,270022,2011-01-25,4,1,1,677,71.085,748.085,e-Shop,1984-04-27,M,9,Clothing,Mens
23050,85115299378,271020,2011-01-25,2,6,4,1052,441.840,4649.840,MBR,1976-06-20,M,8,Home and kitchen,Furnishing
23051,72870271171,270911,2011-01-25,11,5,3,1142,359.730,3785.730,TeleShop,1970-05-22,M,2,Books,Children


In [22]:
ftd=transaction_master_bi.groupby('cust_id').min()['tran_date']
transaction_master_bi['First Purchase Date']=transaction_master_bi.apply(lambda row: ftd.loc[row['cust_id']], axis=1)

In [23]:
#Calculatine the age
today=date.today()
transaction_master_bi['user_age_in_system_month']=(today-transaction_master_bi['First Purchase Date'].dt.date)/np.timedelta64(1,'m')
transaction_master_bi['user_age_in_year']=(today-transaction_master_bi['DOB'].dt.date)/np.timedelta64(1,'D')/365
transaction_master_bi['user_age_in_year']=pd.to_numeric(transaction_master_bi['user_age_in_year'])
tot_txn=transaction_master_bi.groupby('cust_id').count()['transaction_id']
transaction_master_bi['lifetime_txn']=transaction_master_bi.apply(lambda row: tot_txn.loc[row['cust_id']], axis=1)
df=transaction_master_bi[['cust_id','prod_cat','Qty','total_amt']]

In [24]:
#Calculating The Average Price
df['avg_price']=df['total_amt']/df['Qty']
df1=df.groupby(['cust_id','prod_cat']).mean()['avg_price'].to_frame().reset_index()
df1.rename(columns = {'avg_price':'user_aov'}, inplace = True)
transaction_master_bi=pd.merge(left=transaction_master_bi, right=df1,how='left',left_on=['cust_id','prod_cat'],right_on=['cust_id','prod_cat'])


/var/folders/4x/bhr05fqs1xq2l403bnmgyvf00000gn/T/ipykernel_90613/2116177319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_price']=df['total_amt']/df['Qty']


In [25]:
age_ranges = pd.IntervalIndex.from_tuples([(0, 18), (19, 25), (26, 35), (36, 120)])
transaction_master_bi["age_range"] = pd.cut(transaction_master_bi["user_age_in_year"], age_ranges, labels=["<18", "18-25", "25-35", ">35"])

In [26]:
transaction_master_bi["product_code"]=transaction_master_bi['prod_subcat'].astype(str)+transaction_master_bi['prod_cat'].astype(str)

In [27]:
transaction_master_bi

,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type,...,city_code,prod_cat,prod_subcat,First Purchase Date,user_age_in_system_month,user_age_in_year,lifetime_txn,user_aov,age_range,product_code
0,80712190438,270351,2014-02-28,1,1,5,772,405.300,4265.300,e-Shop,...,5,Clothing,Women,2011-12-13,6377760.0,42.367123,5,853.060000,"(36, 120]",WomenClothing
1,29258453508,270384,2014-02-27,5,3,5,1497,785.925,8270.925,e-Shop,...,8,Electronics,Computers,2011-02-17,6808320.0,50.750685,10,1316.791667,"(36, 120]",ComputersElectronics
2,51750724947,273420,2014-02-24,6,5,2,791,166.110,1748.110,TeleShop,...,8,Books,DIY,2012-02-10,6292800.0,31.526027,6,874.055000,"(26, 35]",DIYBooks
3,93274880719,271509,2014-02-24,11,6,3,1363,429.345,4518.345,e-Shop,...,3,Home and kitchen,Bath,2011-02-06,6824160.0,42.668493,6,1506.115000,"(36, 120]",BathHome and kitchen
4,51750724947,273420,2014-02-23,6,5,2,791,166.110,1748.110,TeleShop,...,8,Books,DIY,2012-02-10,6292800.0,31.526027,6,874.055000,"(26, 35]",DIYBooks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23048,94340757522,274550,2011-01-25,12,5,1,1264,132.720,1396.720,e-Shop,...,7,Books,Academic,2011-01-25,6841440.0,51.969863,5,1315.318333,"(36, 120]",AcademicBooks
23049,89780862956,270022,2011-01-25,4,1,1,677,71.085,748.085,e-Shop,...,9,Clothing,Mens,2011-01-25,6841440.0,39.780822,2,748.085000,"(36, 120]",MensClothing
23050,85115299378,271020,2011-01-25,2,6,4,1052,441.840,4649.840,MBR,...,8,Home and kitchen,Furnishing,2011-01-25,6841440.0,47.638356,5,827.645000,"(36, 120]",FurnishingHome and kitchen
23051,72870271171,270911,2011-01-25,11,5,3,1142,359.730,3785.730,TeleShop,...,2,Books,Children,2011-01-25,6841440.0,53.723288,5,1261.910000,"(36, 120]",ChildrenBooks


In [28]:
label_encoder = LabelEncoder()
transaction_master_bi['prod_code'] = label_encoder.fit_transform(transaction_master_bi['product_code'])
transaction_master_bi['sex'] = label_encoder.fit_transform(transaction_master_bi['Gender'])
transaction_master_bi['age_split'] = label_encoder.fit_transform(transaction_master_bi['age_range'])
transaction_master_bi['city'] = label_encoder.fit_transform(transaction_master_bi['city_code'])
edf=transaction_master_bi[['product_code','prod_code']].drop_duplicates()
# Create user-item matrix with additional features
user_item_matrix = pd.pivot_table(transaction_master_bi, values='Qty', index=['cust_id','sex', 'age_split', 'city'],columns='prod_code',aggfunc="sum", fill_value=0,)
user_item_matrix.fillna(0, inplace=True)  # Fill missing values

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
cust_ids = user_item_matrix.index.get_level_values('cust_id').unique()
user_similarity_df = pd.DataFrame(user_similarity, index=cust_ids, columns=cust_ids)


# Function to get recommendations for a given user
def get_recommendations(user_id, user_similarity_matrix, user_item_matrix,top_n=10):
    similar_users = user_similarity_matrix[user_id]
    sm_df=similar_users.to_frame().sort_values(by=[user_id],ascending=False).reset_index()
    sm_df1=sm_df[sm_df[user_id]!=0]
    ranked_users=sm_df1[sm_df1["cust_id"]!=user_id]["cust_id"]
    user_items = user_item_matrix.loc[user_id]
    
    product_scores = {}
    
    for other_user in ranked_users:
        other_user_items = user_item_matrix.loc[other_user]
        x=other_user_items.columns[other_user_items.any()]
        l=other_user_items[0:][list(x)].reset_index()
        for y in x:
            product_scores[y] = product_scores.get(y, 0) + list(l[y])[0]*list(sm_df[sm_df["cust_id"]==other_user][user_id])[0]
    
    keys_to_remove=user_items.columns[user_items.any()]
    #keys_to_remove=list(edf[edf["prod_code"].isin(user_prod_code)]['product_code'])
    filtered_product_scores = {key: value for key, value in product_scores.items() if key not in keys_to_remove}
    s=pd.DataFrame([filtered_product_scores]).transpose()
    s.rename(columns = {0:'score'},inplace=True)
    s2=s.reset_index()
    s2.rename(columns = {'index':'product_code'},inplace=True)
    s1=s.sort_values(by='score',ascending=False)
    df=s1.head(top_n)
    df1=df.reset_index()
    df1.rename(columns={'index':'prod_code'},inplace=True)
    recom=pd.merge(left=df1, right=edf,how='left',left_on=['prod_code'],right_on=['prod_code'])
    recom.drop(columns='prod_code',axis=1,inplace=True)
    return recom
# Example: Get recommendations for user 1
user_id =274609
Recommendations=get_recommendations(user_id, user_similarity_df, user_item_matrix,top_n=10)
recom=Recommendations["product_code"]

print(f"Recommendations for user {user_id}:\n {recom}")


Recommendations for user 274609:
 0              BathHome and kitchen
1                     WomenClothing
2                      FictionBooks
3                         WomenBags
4                      KidsFootwear
5                     ChildrenBooks
6                          MensBags
7    Personal AppliancesElectronics
8                     WomenFootwear
9             ToolsHome and kitchen
Name: product_code, dtype: object


In [29]:
# Example: Get recommendations for user 1
user_id =266783
Recommendations=get_recommendations(user_id, user_similarity_df, user_item_matrix,top_n=15)
recom=Recommendations["product_code"]

In [30]:
print(f"Recommendations for user {user_id}:\n {recom}")

Recommendations for user 266783:
 0              ToolsHome and kitchen
1                      ChildrenBooks
2                      WomenFootwear
3         FurnishingHome and kitchen
4                       KidsClothing
5                       KidsFootwear
6                 MobilesElectronics
7                       FictionBooks
8               BathHome and kitchen
9               ComputersElectronics
10    Personal AppliancesElectronics
11                       ComicsBooks
12                     WomenClothing
13           KitchenHome and kitchen
14        Audio and videoElectronics
Name: product_code, dtype: object


In [31]:

!pip install scikit-surprise

In [32]:
#accuracy check

transaction_master_bi_train,transaction_master_bi_test=train_test_split(transaction_master_bi,test_size=0.25,random_state=42)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

label_encoder = LabelEncoder()
transaction_master_bi_train['prod_code'] = label_encoder.fit_transform(transaction_master_bi_train['product_code'])
transaction_master_bi_train['sex'] = label_encoder.fit_transform(transaction_master_bi_train['Gender'])
transaction_master_bi_train['age_split'] = label_encoder.fit_transform(transaction_master_bi_train['age_range'])
transaction_master_bi_train['city'] = label_encoder.fit_transform(transaction_master_bi_train['city_code'])
edf_train=transaction_master_bi_train[['product_code','prod_code']].drop_duplicates()
# Create user-item matrix with additional features
user_item_matrix_train = pd.pivot_table(transaction_master_bi_train, values='Qty', index=['cust_id','sex', 'age_split', 'city'],columns='prod_code',aggfunc="sum", fill_value=0,)
user_item_matrix_train.fillna(0, inplace=True)  # Fill missing values

# Calculate cosine similarity between users
user_similarity_train = cosine_similarity(user_item_matrix_train)
cust_ids_train = user_item_matrix_train.index.get_level_values('cust_id').unique()
user_similarity_df_train = pd.DataFrame(user_similarity_train, index=cust_ids_train, columns=cust_ids_train)
cust_ids_test=transaction_master_bi_test['cust_id'].unique().tolist()
cust_ids_to_evaluate=list(set(cust_ids_test) & set(cust_ids_train))
actual_test_set = transaction_master_bi_test.groupby('cust_id')['product_code'].agg(list).to_dict()
precision_scores = []
recall_scores = []
f1_scores = []
average_precision_scores = []
for user_id in cust_ids_to_evaluate:
    Recommendations = get_recommendations(user_id, user_similarity_df_train, user_item_matrix_train)
    actual_purchases_for_user =list(set(actual_test_set.get(user_id, [])))
    predicted_purchases_for_user = Recommendations['product_code'].tolist()

    # True labels for the user
    true_labels = [1 if prod_code in actual_purchases_for_user else 0 for prod_code in actual_test_set[user_id]]

    # Predicted scores for the recommendations
    predicted_scores = [1 if prod_code in predicted_purchases_for_user else 0 for prod_code in actual_test_set[user_id]]

    # Calculate precision, recall, F1-score, and average precision
    precision = precision_score(true_labels, predicted_scores)
    recall = recall_score(true_labels, predicted_scores)
    f1 = f1_score(true_labels, predicted_scores)
    average_precision = average_precision_score(true_labels, predicted_scores)

    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    average_precision_scores.append(average_precision)

# Calculate the mean scores across users
mean_precision = sum(precision_scores) / len(precision_scores)
mean_recall = sum(recall_scores) / len(recall_scores)
mean_f1 = sum(f1_scores) / len(f1_scores)
mean_average_precision = sum(average_precision_scores) / len(average_precision_scores)


/Users/shikhasingh/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")
print(f"Mean F1-Score: {mean_f1:.4f}")
print(f"Mean Average Precision: {mean_average_precision:.4f}")